In [0]:
#just the CPU version of FAISS, will have to look deeper on how to get GPU version, but works fast enough for now 

!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

--2019-05-01 19:18:21--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.93.24, 104.17.92.24, 2606:4700::6811:5d18, ...
Connecting to anaconda.org (anaconda.org)|104.17.93.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20190501T191821Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=AgoJb3JpZ2luX2VjEKf%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJHMEUCIQCFn%2FSJ48RNq8%2BZktM8J94LrdqLplOc49ZC%2BcLvgOpdZgIgTMVVysPLBfegOb60Czcn59UdNbWBeShWrLMlHSSSrHQq4wMIkP%2F%2F%2F%2F%2F%2F%

In [0]:
import pandas as pd
import numpy as np
import os
import csv
from tqdm import tqdm
import argparse
from glob import glob
import faiss
from multiprocessing import Pool, cpu_count
from math import ceil

from google.colab import drive
drive.mount('/content/gdrive')

Failed to load GPU Faiss: No module named 'faiss.swigfaiss_gpu'
Faiss falling back to CPU-only.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
class namespace():
  pass

"/content/gdrive/My Drive/FFNNEmbeds"

args = namespace()
args.data_path = '/content/gdrive/My Drive/data/FFNNEmbeds'
args.output_path = '/content/gdrive/My Drive/data/gpt2_train_data/'
args.number_samples = 10
args.batch_size = 512

files = os.listdir(args.data_path)
print(files)

['iClinicFFNNEmbeddings.csv', 'webMDFFNNEmbeddings.csv', 'askDocsFFNNEmbeddings.csv', 'healthTapFFNNEmbeddings.csv', 'eHealthForums.csv']


In [0]:
def fix_array(x):
    x = np.fromstring(
    x.replace('\n','')
    .replace('[','')
    .replace(']','')
    .replace('  ',' '), sep=' ')
    return x.reshape((1, 768))

pd.options.display.max_rows = 700
pd.set_option('expand_frame_repr', True)
pd.set_option('max_colwidth', 250)
pd.get_option("display.max_rows")

qa = pd.read_csv("/content/gdrive/My Drive/data/FFNNEmbeds/" + files[0])
for file in files[1:]:
    print(file)
    qa = pd.concat([qa, pd.read_csv("/content/gdrive/My Drive/data/FFNNEmbeds/" + file)], axis = 0)

del qa['Unnamed: 0']
qa["Q_FFNN_embeds"] = qa["Q_FFNN_embeds"].apply(fix_array)
qa["A_FFNN_embeds"] = qa["A_FFNN_embeds"].apply(fix_array)

qa = qa.reset_index(drop=True)

qa.head()

webMDFFNNEmbeddings.csv
askDocsFFNNEmbeddings.csv
healthTapFFNNEmbeddings.csv
eHealthForums.csv


,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,is it fine to exercise with knee pain?,from your description it appears that you may have anterior knee pain which sometimes presents as pain at the back of the knee. the second possibility is that you have over done your exercise and hamstrings are sore and the lower end of the knee ...,"[[-0.0236410219, 0.128144488, 0.30177781, 0.430800796, 0.0534969121, 1.24597561, 0.908191383, -0.0682020858, 1.49799263, 0.0182549823, 0.159414783, -0.000211991704, 0.000532273552, -0.0226245634, -0.0949951187, -0.0198594704, 1.25414896, 0.360615...","[[-0.115900427, 0.182273567, -0.0992501155, -0.0385564938, 0.740532875, 0.167836308, 0.0228974223, -0.23636511, -0.172942773, 0.25693363, -0.24061957, -0.0263501219, 0.0366276056, -0.126324296, -0.00830298476, 0.158679247, 0.0368386209, -0.393908..."
1,suffering from anxiety restlessness and taking clonazepam & mirtazapine. i had depression & panic attacks for the last 7 years. need a second opinion.,depression anxiety restlessness and panic attacks are best respond to a combination of a selective serotonin reuptake inhibitors (ssris) and benzodiazepines. mirtazapine is not the most efficient ant depressant and it is not a true ssri as fluoxe...,"[[-0.114040159, 0.140852138, 0.0999100953, 0.208959475, 0.445060819, -0.403621316, 0.0552569143, -0.0890201628, 0.380254924, 0.108175397, 0.191386163, 0.198272184, -0.133397803, 0.212966055, 0.352456003, -0.07531479, 0.341809332, 0.151971966, 0.2...","[[-0.155445561, 0.0941457227, 0.0411367081, 0.255660385, 0.30229032, -0.253930658, -0.176501781, 0.917308688, -0.183687523, 0.209650502, -0.0855835676, 0.239034563, 0.762762547, 0.259996265, 0.34940359, -0.0778813362, -0.315282226, -0.212972909, ..."
2,can a thyroid patient eat soybean and fenugreek?,patients with hypothyroidism usually gain weight. your issue is weight loss which is seen in hyperthyroidism. as your thyroid tsh is in the normal range you can continue the same treatment. for gaining weight you need to take food which is adequa...,"[[-0.0945982337, 0.04440815, -0.219836012, 0.196798429, 0.402243942, -0.0414064191, 1.45660985, 0.214540154, 0.410930932, -0.0685333759, 1.27774835, 0.206304967, -0.320117086, 0.0697160363, 0.302823365, -0.181685016, 1.5156101, 0.679543853, -0.08...","[[-0.267755091, 0.496627122, -0.10593427, 0.103044294, 0.482136637, 0.210582703, -0.194033623, 0.417719126, -0.306303591, 0.23062554, -0.251927078, -0.0518328063, 0.22350128, 0.335046589, -0.0103447549, 0.497870088, 0.153194278, -0.197104171, 0.4..."
3,i am not getting my periods after taking fenugreek seeds. why?,fenugreek seed cannot affect your fertility do not worry. a delayed period could be because of stress or hormonal imbalance so please get a urine pregnancy test to rule out pregnancy first. for further queries consult an infertility specialist on...,"[[-0.0772762671, -0.0546492599, -0.290007412, 0.234665141, -0.0717474371, 0.210957736, 0.501454949, -0.33825165, 0.535232365, 0.141104475, 0.105892427, 0.167244852, -0.226345539, 0.0772434399, 0.36760062, -0.121957906, 0.0198328495, 1.51193035, 0...","[[-0.116682649, 0.229011983, 0.0717253983, 0.0508261435, 0.188567623, 0.305067122, -0.131368637, -0.294420958, -0.202981532, 0.0689166486, -0.0496700145, 0.0921775177, 0.522677302, 0.173455626, 0.19158718, -0.0330378525, -0.165551513, -0.17966866..."
4,kindly suggest me a therapy to overcome heat allergy.,your problem is a characteristic of cholinergic urticaria. it is a type of urticaria where patients are allergic to their own sweat. so whenever a patient sweats for example due to heat sun stress exercise. etc. the patient develops hives. levoce...,"[[0.190717205, -0.109118752, 0.225994617, 0.448829651, -0.368060827, 1.09241807, 0.22018972, 0.1560826, -0.269190162, 0.220193535, 1.38247335, 0.249987021, -0.202662915, -0.0893628523, 0.185544237, -0.270314127, 0.238500059, -0.357885599, 0.36902...","[[-0.0469765291, 0.159166887, -0.205679953, -0.120664336, 0.337732494,

In [5]:
#qa = pd.read_hdf(args.data_path, key='qa_embedding')

'''
with Pool(cpu_count()) as p:
    question_bert = p.map(eval, qa["Q_FFNN_embeds"].tolist())
    answer_bert = p.map(eval, qa["A_FFNN_embeds"].tolist())
'''

question_bert = np.concatenate(qa["Q_FFNN_embeds"].values, axis=0)
answer_bert = np.concatenate(qa["A_FFNN_embeds"].values, axis=0)

question_bert = np.array(question_bert)
answer_bert = np.array(answer_bert)

question_bert = question_bert.astype('float32')
answer_bert = answer_bert.astype('float32')

answer_index = faiss.IndexFlatIP(answer_bert.shape[-1])

question_index = faiss.IndexFlatIP(question_bert.shape[-1])

faiss.normalize_L2(question_bert)
faiss.normalize_L2(answer_bert)

answer_index.add(answer_bert)
question_index.add(question_bert)

os.makedirs(args.output_path, exist_ok=True)
output_path = os.path.join(
    args.output_path, 'GPT2_data_FFNN.csv')

output = open(output_path, "w")
writer = csv.writer(output)

firstrow = ['question', 'answer']
for ii in range(0, args.number_samples):
    firstrow.append('question'+str(ii))
    firstrow.append('answer'+str(ii))

writer.writerow(firstrow)

def topKforGPT2(start_ind, end_ind, topk):
    D1, I1 = answer_index.search(
        question_bert[start_ind:end_ind].astype('float32'), topk)
    D2, I2 = question_index.search(
        question_bert[start_ind:end_ind].astype('float32'), topk)
    return I1, I2

steps = ceil(qa.shape[0] / args.batch_size)

# for k in tqdm(range(1000), mininterval=30, maxinterval=60):
for k in tqdm(range(0, qa.shape[0], args.batch_size), total=steps):
    start_ind = k
    end_ind = k+args.batch_size

    a_batch_index, q_batch_index = topKforGPT2(
        start_ind, end_ind, int(args.number_samples/2))
    for a_index, q_index in zip(a_batch_index, q_batch_index):
        rowfill = []
        rowfill.append(qa["question"][k])
        rowfill.append(qa["answer"][k])
        aaa = qa.loc[list(a_index), :]
        qqq = qa.loc[list(q_index), :]
        aaaa = [*sum(zip(list(aaa['question']), list(aaa['answer'])), ())]
        qqqq = [*sum(zip(list(qqq['question']), list(qqq['answer'])), ())]
        finalfill = aaaa+qqqq
        rowfill = rowfill + finalfill

100%|██████████| 468/468 [2:29:09<00:00, 16.79s/it]
